In [14]:
# -*- coding: utf-8 -*-

import numpy as np
import cv2 as cv

L = 256


def getAngledGLCM(img, alpha):
    row, col = img.shape
    #row, col, layer = img.shape
    glcm = np.zeros((L,L), np.uint8)
    xy = [0,0]
    rowStart = 0
    colStart = 0
    rowEnd = row
    colEnd = col - 1
   
    if (alpha == 0):
        xy = [0,1]
    elif (alpha == 45):
        xy = [-1,1]
        rowStart = 1
        colStart = 0
    elif (alpha == 90):
        xy = [-1,0]
        rowStart = 1
        colEnd = col
    else:
        xy = [-1,-1]
        rowStart = 1
        colStart = 1
        colEnd = col
        
    for x in range(rowStart,rowEnd):
        for y in range(colStart,colEnd):
            pixVal = img[x,y]
            pixValNeighbor = img[x + xy[0],y + xy[1]]
            glcm[pixVal,pixValNeighbor] += 1
    
    return glcm / glcm.sum()


def getGLCM(img):
    global glcm
    glcm = np.zeros((L,L), np.float64)
    global alpha 
    alpha = 0
    
    for i in range(4):
        glcm += getAngledGLCM(img, alpha)
        alpha += 45
        
    return glcm / 4


def getSumGLCM(glcm):
    global sumX
    sumX = []
    global sumY
    sumY = []
    
    for i in range(L):
        sumY.append(glcm[:,i].sum())
        sumX.append(glcm[i,:].sum())
        
    return [sumX, sumY]


def getMean(glcm, sumX, sumY):
    #sumX, sumY = getSumGLCM(glcm)
    global meanX
    meanX = 0.0
    global meanY
    meanY = 0.0
    
    for i in range(L):
        meanX += i * sumX[i]
        meanY += i * sumY[i]
        
    return [meanX, meanY]


def getVarianceXY(glcm, sumX, sumY, meanX, meanY):
    #meanX, meanY = getMean(glcm)
    #sumX, sumY = getSumGLCM(glcm)
    global varX
    varX = 0.0
    global varY
    varY = 0.0
    
    for i in range(L):
        varX += ((i - meanX) ** 2) * sumX[i]
        varY += ((i - meanY) ** 2) * sumY[i]
        
    return [varX, varY]


def getStandardDeviation(glcm):
    varX, varY = getVarianceXY(glcm, sumX, sumY, meanX, meanY)
    global sdX
    global sdY
    sdX = np.sqrt(varX)
    sdY = np.sqrt(varY)
    return [np.sqrt(varX), np.sqrt(varY)]


def getASM(glcm):
    return np.power(glcm.flatten(), 2).sum()


def getContrast(glcm):
    con = 0.0
    
    for x in range(L):
        for y in range(L):
            con += ((x - y) ** 2) * glcm[x,y]
            
    return con


def getCorrelation(glcm, meanX, meanY, sdX, sdY):
    cor = 0.0
    #meanX, meanY = getMean(glcm)
    sdX, sdY = getStandardDeviation(glcm)
    
    for x in range(L):
        for y in range(L):
            cor += (x * y) * glcm[x,y]
            
    return (cor - (meanX * meanY)) / (sdX * sdY)


def getVariance(glcm, meanX, meanY):
    #meanX, meanY = getMean(glcm)
    global var
    var = 0.0
    
    for x in range(L):
        for y in range(L):
            var += (x - meanX) * (y - meanY) * glcm[x,y]
            
    return var


def getIDM(glcm):
    idm = 0.0
    
    for x in range(L):
        for y in range(L):
            idm += glcm[x,y] / (1 + ((x - y) ** 2))
            
    return idm


def getEntropy(glcm):
    E = 0.0
    
    for x in range(L):
        for y in range(L):
            if (glcm[x,y] <= 0.0):
                E += glcm[x,y] * np.log2(np.e)
            else:
                E += glcm[x,y] * np.log2(glcm[x,y])
                
    return -E

def getHomogeneity(glcm):
    '''
    Calculate GLCM Homogeneity
    '''
    homo = 0.0
    for i in range(L):
        for j in range(L):
            homo += glcm[i,j] / (1.+(i-j)**2)

    return homo

def getEnergy(glcm):
    '''
    Calculate GLCM Energy
    '''    
    asm = 0.0
    for i in range(L):
        for j in range(L):
            asm  += glcm[i,j]**2

    ene = np.sqrt(asm)
    return ene

def getDissimilarity(glcm):
    '''
    Calculate GLCM Dissimilarity
    '''
    diss = 0.0
    for i in range(L):
        for j in range(L):
            diss += glcm[i,j] * np.abs(i-j)

    return diss
   
img = cv.cvtColor(cv.imread(r"C:\Users\USER\Desktop\3.jpg"), cv.COLOR_BGR2GRAY)
#print("GLCM:",getGLCM(img))
#print("Angled GLCM:", getAngledGLCM(img,alpha))
#print("Sum GLCM:", getSumGLCM(glcm))
#print("Mean:", getMean(glcm, sumX, sumY))
print("Variance:", getVariance(glcm, meanX, meanY))
#print("Standard Deviation:",getStandardDeviation(glcm))
print("ASM:",getASM(glcm))
print("Contrast:",getContrast(glcm))
print("Correlation:",getCorrelation(glcm, meanX, meanY, sdX, sdY))
print("IDM:", getIDM(glcm))
print("Entropy:",getEntropy(glcm))
print("Homogenity:",getHomogeneity(glcm))
print("Energy:", getEnergy(glcm))
print("Dissimilarity:", getDissimilarity(glcm))

Variance: 818963.1671575019
ASM: 0.1499528279895016
Contrast: 6142.374896772191
Correlation: -0.3357820637023342
IDM: 1.758979673413674
Entropy: 24.899196703710892
Homogenity: 1.758979673413674
Energy: 0.38723743102843433
Dissimilarity: 44.87344183988037
